## Programming Assignment 03

Student ID: 1151277

Collaborators: 919755, 1153929, 1152810, 1151248

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from numpy.linalg import matrix_power as mp

from statsmodels.tsa.api import VAR

#### Helper Functions from previous assignments

In [2]:
def Z_matrix(y: np.array, p: int, c: int):
    """Calculate the Z-matrix for a given input

    Args:
        y (np.array): input with all the data of shape (T + p) × K
        p (int): lags
        c (int): intercept yes=1, no=0

    Returns:
        (np.array): Z-matrix for given input
    """

    y = y.T

    #determine matrix dimensions:
    T = y.shape[1] - p
    K = y.shape[0]

    # build Z-matrix
    if c==1:
        Z = np.ones((1, T+p), dtype=float)

    # 1b stacked lagged data
    for i in range(p):
        #add i columns of leading zeros (EDIT: empty, comp cost lower) to ktpmat
        zeros = np.zeros((K, i), dtype=float)
        zerostack = np.hstack((zeros, y[:,:(T+p-i)]))
        # vertically stack this to Z
        Z = np.vstack((Z, zerostack))

    # cutting of leading p columns and retrieving Z
    Z = Z[:, p-1:-1]

    return Z

In [3]:
def B_matrix(y: np.array, p: int, c: int):
    """Calculates the B matrix with the estimated coefficients

    Args:
        y (np.array): input with all the data of shape (T + p) × K
        p (int): lags
        c (int): intercept yes=1, no=0

    Returns:
        _type_: B = matrix with estimated coefficients; Z=Z-matrix; sigma_u=covariance matrix
    """

    # get Z-matrix from function above
    Z = Z_matrix(y, p, c)

    y = y.T # transpose y
    y = y[:,p:] # first p observations are lost as we need prior lags for estimation
    K = y.shape[0] # number of variables
    T = y.shape[1] # number of observations

    # calculate B
    B = y @ Z.T @ np.linalg.inv((Z@Z.T))

    # calculate sigma_u (covariance matrix)
    sigma_u = (1/(T-K*p-1))*(y-(B@Z))@(y-(B@Z)).T

    return B, Z, sigma_u

#### Exercise 1

In [4]:
# helperfunction
def resid_bootstrap(y,T,K,B,Z, p):
    '''
    helperfunction used to calculate B, Z and sigma_u when applying the bootstrapping later
    :param y: a K x T + p matrix of observations on yt,
    :param T: number of observations
    :param K: number of variables
    :param B: B matrix with estimated coefficients
    :param Z: Z matrix
    :param p: the lag length p,
    :return: estimations for B, Z and sigma_u for one bootstrap iteration
    '''

    '''
    (2) Centered residuals are computed (usual average). Bootstrap residuals u∗1, . . . , u∗T are then obtained by randomly drawing with replacement from the centered residuals.
    '''

    uthat = y[:,p:] - (B@Z)
    uthatbar = np.sum(uthat, axis=1)/T
    uthatcenterded = uthat - uthatbar.T.reshape(K, 1)
    draws = np.random.randint(0, T, T)


    '''
    (3) Bootstrap time series are computed recursively [...]where the same initial values may be used for each generated series, (y∗ −p+1, . . . , y∗0) = (y−p+1, . . . , y0).
    '''
    # set bootstrap time series pre-sample values to the same presample series from original data for every repetition

    bs_y = y[:,:p]

    for i in range(T):
        y_t = B[:,0] + uthatcenterded[:,draws[i]]
        for l in range(p):
            y_t = y_t + (B[:, (l*K+1):(l*K+K+1)] @ bs_y[:,-(l+1)])  
        bs_y = np.hstack((bs_y, y_t.reshape(K, 1)))

    B_bs, Z_bs, sigma_u_bs = B_matrix(bs_y.T, p, c=1)

    return B_bs, Z_bs, sigma_u_bs

In [5]:
# bootstrapping function
def bootstrap_se(Tpkmat, p, R):
    """
    Tpkmat:a T + p × K matrix of observations on yt
    p:the lag length
    R: and the number of bootstrap replications
    returns the bootstrap standard error of VAR coefficients in B 
    """
    y = Tpkmat.T # transpose input matrix to K x (T+p)
    T = y.shape[1] - p # get T (number of observations)
    K = y.shape[0]

    '''
    Description from Lütkepohl, appendix D, page 709
    (1) The parameters of the model under consideration are estimated. Let uthat, t = 1, . . . , T, be the estimation residuals.
    '''

    B, Z, sigma_u = B_matrix(Tpkmat, p, c=1)


    
    # apply bootstrapping using helper function from above
    for i in range(R):
        if i == 0:
            B_bs_list, _, _ = resid_bootstrap(y,T,K,B,Z, p)
        else:
            B_bs, _, _ = resid_bootstrap(y,T,K,B,Z, p)
            B_bs_list = np.dstack((B_bs_list, B_bs))
    
    #calculate standard error
    Bbar_bs_list = np.mean(B_bs_list, axis = 2)
    deviation = B_bs_list - Bbar_bs_list[:, :, None]
    deviation_squared = deviation**2
    sd = np.sqrt(np.sum(deviation_squared, axis=2)/(R-1))

    return sd.T

#### Exercise 2

In [6]:
# read in data
awm = pd.read_csv("awm19up18.csv")
awm.rename(columns={awm.columns[0]: "Q" }, inplace = True)

of_interest = ["Q", "YER", "ITR", "LTN", "STN"]
awm = awm[awm.columns.intersection(of_interest)]
awm.set_index('Q', inplace=True)

In [7]:
# calculate logs and first differences and assign names accordingly
awm["YER_log"] = np.log(awm['YER'])
awm["ITR_log"] = np.log(awm['ITR'])

awm["d_lgdp"] = awm["YER_log"].diff()
awm["d_invest"] = awm["ITR_log"].diff()

awm["d_lgdp"] = awm["d_lgdp"] * 400
awm["d_invest"] = awm["d_invest"] * 400

awm["d_R"] = awm["LTN"].diff()
awm["d_r"] = awm["STN"].diff()

awm.dropna(inplace=True)

In [8]:
# get the input for our function
y_t = np.array(awm[["d_lgdp", "d_invest", "d_R", "d_r"]])

In [9]:
# test our function
B, Z, sigma_u = B_matrix(y_t, p=2, c=1)
B_se = bootstrap_se(y_t, 2, R=499)

In [10]:
# show our B_se
print(B_se)

[[0.30425759 0.73592353 0.04279562 0.06816006]
 [0.10002155 0.25754394 0.01404188 0.02396775]
 [0.03856782 0.09594441 0.00597267 0.00995316]
 [0.54897577 1.41293458 0.08084083 0.12250975]
 [0.34146842 0.8693739  0.04951256 0.07961024]
 [0.10022499 0.25092225 0.01623496 0.02512259]
 [0.03718894 0.08926193 0.00607914 0.00881139]
 [0.53032229 1.33326117 0.08121617 0.13284352]
 [0.31073515 0.80000332 0.04762424 0.07529631]]


In [11]:
# Compare to built-in function from VAR package
model = VAR(y_t)
var = model.fit(2) #number = lag order
var.bse # standard errors

array([[0.2697177 , 0.6727813 , 0.04033584, 0.0646388 ],
       [0.09634184, 0.24031419, 0.01440777, 0.02308866],
       [0.03812077, 0.09508808, 0.0057009 , 0.00913578],
       [0.53710206, 1.3397423 , 0.08032274, 0.12871841],
       [0.33948962, 0.8468197 , 0.05077012, 0.08135989],
       [0.10218602, 0.25489185, 0.01528175, 0.02448924],
       [0.03699542, 0.09228102, 0.0055326 , 0.00886608],
       [0.54350851, 1.3557225 , 0.08128081, 0.13025374],
       [0.32538508, 0.81163747, 0.04866081, 0.07797969]])

In [12]:
# show whole built-in output 
#var.summary()

#### Exercise 3

In [13]:
def var2sim(A1: np.array, A2: np.array, sigma_u: np.array, T: int):
    """A function that simulates time series data from a K-dimensional VAR(2) process yt = A1 y_t−1 + A2 y_t−2 + u_t, 
    where the innovations ut are drawn from a multivariate normal distribution with mean zero and covariance matrix Σ_u. 
    Uses y_−1 = y_0 = 0 as starting values, where 0 is a K ×1 vector of zeros.
    Generates time series of length T+50 and discards the first 50 observations, such that it returns a time series of total length equal to T.

    Args:
        A1 (np.array): coefficient matrix at lag 1
        A2 (np.array): coefficient matrix at lag 2
        sigma_u (np.array): covariance matrix Σ_u
        T (int): number of observations

    Returns:
        np.array: T x K matrix of observations on y_t
    """
    K = sigma_u.shape[0]

    # set starting values
    y_tminus1 = np.zeros((K, 1))
    y_tminus2 = np.zeros((K, 1))

    P = np.linalg.cholesky(sigma_u)

    for i in range(T+50):
        # draw disturbance u_t
        u_t = P @ np.random.standard_normal(K)
        u_t = u_t.reshape(K, 1)
        #recursively calculate y_t 
        y_t = A1@y_tminus1 + A2@y_tminus2 - 2 + u_t
        if i == 0:
            y = y_t
        else:
            y = np.hstack((y, y_t))
        y_tminus2 = y_tminus1
        y_tminus1 = y_t

    # discard first 50 observations
    y = y[:,50:]

    return y.T

In [14]:
# Test our var2sim function

K = 4
p = 2
T = 100

A1 = B[:,1:K+1]
A2 = B[:,K+1:2*K+1]

var2sim(A1, A2, sigma_u, T)

array([[ 2.28971509e+00,  1.55043403e+01, -3.52301237e+00,
        -3.01881083e+00],
       [-9.57069986e-01,  1.32301899e+01, -3.90374498e+00,
        -3.50761557e+00],
       [-1.63443569e+00,  3.80784070e+00, -3.23163154e+00,
        -3.95836158e+00],
       [ 1.32698380e-01,  7.18128084e+00, -3.37339436e+00,
        -4.14400849e+00],
       [ 2.01951392e+00,  1.09452270e+01, -3.57049678e+00,
        -2.44118585e+00],
       [ 3.28337165e+00,  2.15674529e+01, -3.46502658e+00,
        -2.75677552e+00],
       [ 2.71629648e+00,  1.72853498e+01, -3.20029610e+00,
        -3.18901292e+00],
       [ 9.27610515e-01,  1.65433744e+01, -3.16081049e+00,
        -2.88865798e+00],
       [ 2.73123063e+00,  1.12577330e+01, -3.18966837e+00,
        -2.53708950e+00],
       [ 3.79696187e+00,  1.24905425e+01, -3.36350744e+00,
        -2.94686662e+00],
       [-1.62530776e+00,  2.60959583e+00, -3.17012215e+00,
        -3.02413410e+00],
       [ 4.30665747e-02,  7.47925997e+00, -3.89470104e+00,
      

#### Exercise 4

In [15]:
def hstep_forecast(y: np.array, p: int, h: int):
    """A function that computes the h-step ahead point forecasts y_T (h) and the corresponding MSE matrix Σˆ_y(h) based on a VAR(p) with intercept

    Args:
        y (np.array): K × T matrix of observations
        p (int): lag order
        h (int): forecast horizon

    Returns:
        list: h-step ahead forecasts and the corresponding MSE matrix
    """
    
    K = y.shape[0]
    T = y.shape[1]

    # retrieving estimates
    B, Z, sigma_u = B_matrix(y.T, p, c=1)

    # constructing matrices
    J1 = np.hstack((np.zeros((K, 1)), np.identity(K), np.zeros((K, K*(p-1)))))

    row0 = np.hstack((np.ones((1,1)), np.zeros((1, K*p))))
    rowz = np.hstack((np.zeros((K*(p-1), 1)), np.identity(K*(p-1)), np.zeros((K*(p-1), K))))
    B = np.vstack((row0, B, rowz))

    Zt = y[:,-p:]                       # selecting y[:,-p:] from t-p up to t
    Zt = Zt[:,::-1].T.flatten()             # reverse order horizontally, transpose and flatten.
    Zt = np.hstack((np.array([(1)]), Zt)).reshape(K*p+1, 1)   # adding one leading 1, transposing, dimension is: 1+K*T x 1
   
     # predicting y_th
    y_th = J1@mp(B, h)@Zt
    
    # calculate the corresponding MSE matrix
    sigma_hat_yh = 0
    for i in range(h):                # formula at p. 64
        PHIi = J1@mp(B, i)@J1.T 
        part_of_sum = PHIi@sigma_u@PHIi.T
        sigma_hat_yh += part_of_sum

    return y_th, sigma_hat_yh

#### Exercise 5

In [16]:
# Define parameters

T = 100
p = 2

A1 = np.array([(0.4, 0.25), (0.0, 0.5)])
A2 = np.array([(0.2, 0.4), (0.0, 0.0)])
sigma_u = np.array([(1, 0.5), (0.5, 1)])

In [17]:
# Test functions

time_series_TK = var2sim(A1, A2, sigma_u, T)

In [18]:
# Forecast horizon h = 1
h = 1
y_th1, mse_mat1 = hstep_forecast(time_series_TK.T, p, h)

print(y_th1) 
mse_mat1

[[-13.62604712]
 [ -4.11882076]]


array([[1.06892064, 0.54797763],
       [0.54797763, 1.00714966]])

In [19]:
# Forecast horizon h = 4
h = 4
y_th4, mse_mat4 = hstep_forecast(time_series_TK.T, p, h)
print(y_th4) 
mse_mat4

[[-12.53209079]
 [ -4.35780055]]


array([[3.02534847, 1.27699281],
       [1.27699281, 1.61979859]])

In [20]:
# set up 95 % interval forecast (assuming data is generated from gaussian process)

CIone95 = [y_th1.reshape(2,) - 1.96 * np.sqrt(np.diag(mse_mat1)), y_th1.reshape(2,) + 1.96 * np.sqrt(np.diag(mse_mat1))]
CIfour95 = [y_th4.reshape(2,) - 1.96 * np.sqrt(np.diag(mse_mat4)), y_th4.reshape(2,) + 1.96 * np.sqrt(np.diag(mse_mat4))]

# note that resulting confidence have the for: [array of negative intervall bound for both k, array of positive intervall bound for both k]

In [21]:
# Compare to built-in function from VAR package
model = VAR(time_series_TK)
results = model.fit(2)

# result for h = 1
print(results.forecast(time_series_TK, 1))
results.mse(1)

[[-13.62604712  -4.11882076]]


array([[[1.06892064, 0.54797763],
        [0.54797763, 1.00714966]]])

In [22]:
# result for h = 4
print(results.forecast(time_series_TK, 4))
results.mse(4)

[[-13.62604712  -4.11882076]
 [-13.16055237  -4.18331592]
 [-12.72339058  -4.28157107]
 [-12.53209079  -4.35780055]]


array([[[1.06892064, 0.54797763],
        [0.54797763, 1.00714966]],

       [[1.49135162, 0.89063576],
        [0.89063576, 1.48543085]],

       [[2.35096243, 1.18990685],
        [1.18990685, 1.60719652]],

       [[3.02534847, 1.27699281],
        [1.27699281, 1.61979859]]])